<a href="https://colab.research.google.com/github/amandafanny/langChain_colab/blob/main/langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 安装环境
OPENAI_API_KEY = "" #@param {type:"string"}
!pip install langchain
!pip install openai

import os
%env
# %reload_ext autoreload
os.environ["OPENAI_API_KEY"]=""

# Models

## LLMs

In [ ]:
#@title 异步处理
import time
import asyncio

from langchain.llms import OpenAI

def generate_serially():
    llm = OpenAI(temperature=0.9)
    for _ in range(10):
        resp = llm.generate(["Hello, how are you?"])
        print(resp.generations[0][0].text)


async def async_generate(llm):
    resp = await llm.agenerate(["Hello, how are you?"])
    print(resp.generations[0][0].text)


async def generate_concurrently():
    llm = OpenAI(temperature=0.9)
    tasks = [async_generate(llm) for _ in range(10)]
    await asyncio.gather(*tasks)


s = time.perf_counter()
# If running this outside of Jupyter, use asyncio.run(generate_concurrently())
await generate_concurrently() 
elapsed = time.perf_counter() - s
print('\033[1m' + f"Concurrent executed in {elapsed:0.2f} seconds." + '\033[0m')

s = time.perf_counter()
generate_serially()
elapsed = time.perf_counter() - s
print('\033[1m' + f"Serial executed in {elapsed:0.2f} seconds." + '\033[0m')

In [5]:
#@title How to write a custom LLM wrappe

#@markdown There is only one required thing that a custom LLM needs to implement:

#@markdown 1. A `_call` method that takes in a string, some optional stop words, and returns a string

#@markdown There is a second optional thing it can implement:

#@markdown 1. An `_identifying_params` property that is used to help with printing of this class. Should return a dictionary.

from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

class CustomLLM(LLM):
    
    n: int
        
    @property
    def _llm_type(self) -> str:
        return "custom"
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        return prompt[:self.n]
    
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"n": self.n}

llm = CustomLLM(n=5)
print(llm("This is a foobar thing"))
print(llm)


CustomLLM
Params: {'n': 5}


In [15]:
from langchain.llms.fake import FakeListLLM

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = load_tools(["python_repl"])

responses=[
    "Action: Python REPL\nAction Input: print(2 + 2)",
    "Final Answer: 4"
]
llm = FakeListLLM(responses=responses)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.run("whats 2 + 2")



> Entering new AgentExecutor chain...
Action: Python REPL
Action Input: print(2 + 2)
Observation: 4

Thought:Final Answer: 4

> Finished chain.


'4'

### How to cache LLM calls

#### In Memory Cache

In [32]:
from langchain.llms import OpenAI

import langchain
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

# To make the caching really obvious, lets use a slower model.
llm = OpenAI(model_name="text-davinci-002", n=2, best_of=2)

In [33]:
%%time
# The first time, it is not yet in cache, so it should take longer
llm("Tell me a joke")

CPU times: user 12.8 ms, sys: 0 ns, total: 12.8 ms
Wall time: 876 ms


'\n\nTwo guys stole a calendar. They got six months each.'

In [34]:
%%time
# The second time it is, so it goes faster
llm("Tell me a joke")

CPU times: user 194 µs, sys: 0 ns, total: 194 µs
Wall time: 200 µs


'\n\nTwo guys stole a calendar. They got six months each.'

#### SQLite

In [35]:
!rm .langchain.db

# We can do the same thing with a SQLite cache
from langchain.cache import SQLiteCache
langchain.llm_cache = SQLiteCache(database_path=".langchain.db")


In [36]:
%%time
# The first time, it is not yet in cache, so it should take longer
llm("Tell me a joke")

CPU times: user 22.1 ms, sys: 3.65 ms, total: 25.8 ms
Wall time: 1.39 s


'\n\nWhy did the chicken cross the road?\n\nTo get to the other side.'

In [37]:
%%time
# The second time it is, so it goes faster
llm("Tell me a joke")

CPU times: user 1.36 ms, sys: 973 µs, total: 2.33 ms
Wall time: 2.34 ms


'\n\nWhy did the chicken cross the road?\n\nTo get to the other side.'

#### Redis Cache

In [39]:
!pip install redis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 8.5 MB/s eta 0:00:00


In [43]:
# We can do the same thing with a Redis cache
# (make sure your local Redis instance is running first before running this example)
from redis import Redis
from langchain.cache import RedisCache
langchain.llm_cache = RedisCache(redis_=Redis())

In [ ]:
%%time
# The first time, it is not yet in cache, so it should take longer
llm("Tell me a joke")

In [ ]:
%%time
# The second time it is, so it goes faster
llm("Tell me a joke")